In [ ]:
from itertools import islice
import Levenshtein

Define functions to be used later.

In [ ]:
# stolen from https://gist.github.com/jakebiesinger/759018/1b7d6bd6967780a8bbae743760c37885bdf86467
def readFastq(fastqfile):
    "parse a fastq-formatted file, yielding a (header, sequence, quality) tuple"
    fastqiter = (l.strip('\n') for l in fastqfile)  # strip trailing newlines 
    fastqiter = filter(lambda l: l, fastqiter)  # skip blank lines
    while True:
        fqlines = list(islice(fastqiter, 4))
        if len(fqlines) == 4:
            header1,seq,header2,qual = fqlines
        elif len(fqlines) == 0:
            return
        else:
            raise EOFError("Failed to parse four lines from fastq file!")

        if header1.startswith('@') and header2.startswith('+'):
            yield header1[1:], seq, qual
        else:
            raise ValueError("Invalid header lines: %s and %s for seq %s" % (header1, header2, seq))

def hamming(a, b):
  return sum(i != j for i, j in zip(a, b))

def rev_comp(seq):
    comp = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    return "".join(comp[base] for base in seq[::-1])

def all_kmers(seq, k):
    return [seq[i:i + k] for i in range(len(seq) - k + 1)]

# all_kmers('ACGTACGTACGT', 4)

Get reads from fastq

In [ ]:
with open('20221021_plasmidsaurus/Lab_j3x_1_ligation_barcode_no_PEG.fastq', 'r') as f:
  no_PEG = readFastq(f)
  no_PEG_reads = [seq for _, seq, _ in no_PEG]

with open('20221021_plasmidsaurus/Lab_j3x_2_ligation_barcode_PEG.fastq', 'r') as f:
  with_PEG = readFastq(f)
  PEG_reads = [seq for _, seq, _ in with_PEG]

# print(no_PEG_reads[:5])
# print(PEG_reads[:5])
print(len(no_PEG_reads))
print(len(PEG_reads))

Concatenate each read to itself in case the plasmid was linearized near the insertion site.

In [ ]:
no_PEG_reads_concat = [i + i for i in no_PEG_reads]
PEG_reads_concat = [i + i for i in PEG_reads]

# print(len(PEG_reads[0]))
# print(len(PEG_reads_concat[0]))

Search for the sequence that directly precedes the insertion site ('CCCGGGTACCGAGCTCGAATTCCCAATACT'). If it isn't found, then search the reverse complement. Take the next 200 bases and search for the sequence that comes after the insertion site. If both are found, then output the upstream site, insertion, and downstream site.

In [ ]:
def first_index_of_kmer(seq, kmer, dist, dist_func):
  k = len(kmer)
  kmers = all_kmers(seq, k)
  for i, s in enumerate(kmers):
    if dist_func(kmer, s) <= dist:
      return i
  return -1

def best_index_of_kmer(seq, kmer, dist, dist_func):
  k = len(kmer)
  kmers = all_kmers(seq, k)
  lowest_distance = dist + 1
  best_index = -1
  for i, s in enumerate(kmers):
    distance = dist_func(kmer, s)
    if distance <= dist and distance < lowest_distance:
      lowest_distance = distance
      best_index = i
  return best_index

def get_inserts(seqs, upstream, downstream, dist, dist_func):
  insert_regions = []
  for seq in seqs:
    index = first_index_of_kmer(seq, upstream, dist, dist_func)
    if index >= 0:
      insert_regions.append(seq[index:index + 200])
    else:
      index_rev = first_index_of_kmer(rev_comp(seq), upstream, dist, dist_func)
      if index_rev >= 0:
        insert_regions.append(rev_comp(seq)[index_rev:index_rev + 200])
  inserts = []
  for seq in insert_regions:
    up_idx = best_index_of_kmer(seq, upstream, dist, dist_func)
    down_idx = best_index_of_kmer(seq, downstream, dist, dist_func)
    if down_idx >= 0:
      inserts.append((seq[up_idx:up_idx + len(upstream)], seq[up_idx + len(upstream): down_idx], seq[down_idx:down_idx + len(downstream)]))

  max_width = max([len(insert) for up, insert, down in inserts])
  table = ["{:<30}\t{:<{width}}\t{:<30}".format(*['Upstream', 'Insert', 'Downstream'], width=max_width)]
  for upstream, insert, downstream in inserts:
    if len(insert) > 0:
      table.append("{:<30}\t{:<{width}}\t{:<30}".format(upstream, insert, downstream, width=max_width))
      table.append("{:<30}\t{:<{width}}\t{:<30}".format("", rev_comp(insert), "", width=max_width))
  print(f'Number of matches found: {len(inserts)}')
  print('\n'.join(table))

  return inserts

In [ ]:
get_inserts(PEG_reads_concat, 'CCCGGGTACCGAGCTCGAATTCCCAATACT', 'AGTATTGGGAATTCACTGGCCGTCGTTTTA', 9, Levenshtein.distance)


In [ ]:
get_inserts(no_PEG_reads_concat, 'CCCGGGTACCGAGCTCGAATTCCCAATACT', 'AGTATTGGGAATTCACTGGCCGTCGTTTTA', 9, Levenshtein.distance)